In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
results_folder = r'../example/results/double_mass'
stations_csv_file = r'../example/data/Stations.csv'
precipitations_daily_ts_parquet = r'precipitations_daily_time_series.parquet'

if not os.path.exists(results_folder):
    os.mkdir(results_folder)

In [3]:
df = pd.read_parquet(precipitations_daily_ts_parquet)
df_stations = pd.read_csv(stations_csv_file)

In [4]:
stations_dict = { x['Id'] : x['Name']  for x in df_stations.to_dict(orient='records')}
df['StationName'] = df['StationId'].apply(lambda x: stations_dict[x])

In [5]:
df_pivot_day = pd.pivot_table(df[df['IR']>2], values='IR', index=['DateTime'], columns=['StationId'], aggfunc=np.sum)
df_pivot_day.index = pd.to_datetime(df_pivot_day.index)

In [6]:
# Get statios statistics
df.groupby('StationId').agg({'DateTime': ['min', 'max'], 'IR': 'max'})

DateTime                      IR
                  min         max         max
StationId                                    
Station1   2015-09-01  2017-10-26   61.599976
Station2   2009-10-05  2017-10-26  453.150140
Station3   2005-06-01  2017-10-26  754.187500
Station4   2005-01-01  2017-10-26  261.400390
Station5   2015-05-12  2017-10-26   62.199950
Station6   2009-12-31  2017-10-26  351.899994
Station7   2003-11-19  2017-10-26  205.400390
Station8   2014-07-14  2017-03-11   67.000000

In [7]:
df_pivot_day_filtered = df_pivot_day[df_pivot_day.index > '2015-09-01'].copy()

In [8]:
df_pivot_day_filtered['average'] = df_pivot_day_filtered.mean(axis=1)

In [9]:
df_pivot_day_filtered.reset_index(inplace=True)

In [10]:
def get_double_mass_graph(df_in, img_filename):
    import plotly.graph_objs as go
    import numpy as np
    from plotly.subplots import make_subplots
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf',
          '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
    all_results = []
    c = 0
    #-------------- Plot preparation
    col_title = df_in.columns[0]
    ncol = 1
    nf = ncol-1
    nfig = 0
    for i in range(1, len(df_in.columns)):
        nfig += i
    rows = int(nfig/ncol+(1 if nfig%ncol>0 else 0))
    fig = make_subplots(rows=rows, cols=ncol)
    fig.update_layout(title='Double Mass Curves', title_x=0.5, titlefont=dict(size=14), margin=dict(t=25, b=15, r=15, l=15),  
                    width=500, height=rows*500, showlegend=False, colorway=colors)
    #-------------- Excel file creation for outputs
    file_out= os.path.join(results_folder, col_title + " - Double Mass Results.xlsx")
    writer=pd.ExcelWriter(file_out)
    workbook  = writer.book
    #-------------- Data treatment
    for col1 in df_in.columns[:len(df_in.columns)-1]:# Loop under each Time Serie
        c += 1
        for col2 in df_in.iloc[:, c:]:              # Loop under the following time series
            df = df_in[ (~np.isnan(df_in[col1])) & (~np.isnan(df_in[col2])) ]# Mask for the datafram to eliminate missing years
            cumul = df.cumsum()# Double Mass Dataframe for each couple of Time Series
            #------- Search the break point 
            coef1 = []
            coef2 = []
            res1=[]
            res2=[]
            index = df.index[2:-2]
            for an in index[:]:# Loop under each row of the Double Mass Dataframe
                #print(an)
                coef1.append(list(np.polyfit( cumul.loc[:an][col1], cumul.loc[:an][col2], 1, full=True)[0]))
                coef2.append(list(np.polyfit( cumul.loc[an:][col1], cumul.loc[an:][col2], 1, full=True)[0]))
                res1.append(np.polyfit( cumul.loc[:an][col1], cumul.loc[:an][col2], 1, full=True)[1][0])
                res2.append(np.polyfit( cumul.loc[an:][col1], cumul.loc[an:][col2], 1, full=True)[1][0])
            index_break = pd.DataFrame(np.sum(np.array([res1, res2]), 0), index=index[:], columns=['residual']).idxmin()
            coef_per1 = pd.DataFrame(np.array(coef1), index=index, columns=['a', 'b']).loc[index_break]
            coef_per2 = pd.DataFrame(np.array(coef2), index=index, columns=['a', 'b']).loc[index_break]
            index_break = index_break[0]

            #display(pd.DataFrame(np.array(coef1), index=index, columns=['a', 'b']))
            coef = pd.concat([coef_per1, coef_per2])
            coef = pd.DataFrame(coef.values, index=['Before '+str(index_break), 'After '+str(index_break)],
                            columns=['a', 'b'])
            name = col1+' - '+col2
            # display(name, coef)
            all_results.append([[col1, col2], index_break, coef])# store result for F test
        #------- Save output (Double Mass Table) to Excel file              
            coef.to_excel(writer, sheet_name='Double Mass Tables', startrow=1, startcol=1+(nf-ncol+1)*3-1)
            cumul[[col1, col2]].to_excel(writer, sheet_name='Double Mass Tables', startrow=5, startcol=1+(nf-ncol+1)*3-1)        
            worksheet = writer.sheets['Double Mass Tables']
            worksheet.cell(row=1, column=1).value = 'Linear Regression Coefficients (a=slope, b=ordinate at origin)'
            worksheet.cell(row=1, column=5).value = 'Double Mass Tables'
            
            #------- Plot  
            nf += 1
            go.Figure()
            fig.update_xaxes(title_text=col1, row=int(nf/ncol), col=nf%ncol+1, range=[0, cumul[col1].max()], titlefont=dict(size=12) )
            fig.update_yaxes(title_text=col2, row=int(nf/ncol), col=nf%ncol+1, range=[0, cumul[col2].max()], titlefont=dict(size=12) )
            fig.add_trace(go.Scatter(x=cumul[col1], y=cumul[col2], text=cumul.index, name="Double mass", mode='markers', 
                                    marker=dict(symbol=100, size=6, color='blue')), row=int(nf/ncol), col=nf%ncol+1)
            fig.add_trace(go.Scatter(x=[0, cumul[col1].max()], y=[0, cumul[col2].max()], name="Straight line", mode='lines',
                            line=dict(color='grey', width=1, dash='dash')), row=int(nf/ncol), col=nf%ncol+1)
            x = cumul[col1][index_break]
            fig.add_trace(go.Scatter(x=[x], y=[cumul[col2][index_break]], text=str(index_break), textposition='middle right', 
                                    name='Break Point', mode='markers+text', marker=dict(symbol=101, size=7, color='red')), 
                                    row=int(nf/ncol), col=nf%ncol+1)
            y = coef.iloc[0][1] + coef.iloc[0][0]*x
            fig.add_trace(go.Scatter(x=[0,  x], y=[coef.iloc[0][1], y], mode='lines', name='Linear 1', 
                                    line=dict(color='black', width=1, dash='solid')), row=int(nf/ncol), col=nf%ncol+1)
            y = coef.iloc[1][1] + coef.iloc[1][0]*x
            x2 = cumul[col1].max()
            y2 = coef.iloc[1][1] + coef.iloc[1][0]*x2
            fig.add_trace(go.Scatter(x=[x,  x2], y=[y, y2], mode='lines', name='Linear 2', 
                                    line=dict(color='black', width=1, dash='solid')), row=int(nf/ncol), col=nf%ncol+1)
    
    fig.show()
    fig.write_image(file=img_filename, format='png')
    writer.close()

In [11]:
for col in df_pivot_day_filtered.columns:
    if col in ('DateTime','average'): continue
    print(stations_dict[col])
    df_in = df_pivot_day_filtered[['DateTime', col, "average"]].dropna().set_index('DateTime').rename(columns = {col: stations_dict[col]})
    img_filename = os.path.join(results_folder, '{0}_double_mass.png'.format(stations_dict[col]))
    get_double_mass_graph(df_in, img_filename)

Station1 Name


Station2 Name


Station3 Name


Station4 Name


Station5 Name


Station6 Name


Station7 Name


Station8 Name
